In [37]:
# Analyze games and save evaluation for each move

In [55]:
import chess #https://github.com/niklasf/python-chess
from stockfish import Stockfish

#We'll create an intentionally worse stockfish engine to estimate complexity of positions
stockfish_good=Stockfish("C:/Users/aober/Documents/Data_Science_Studium/4Semester/BigData/stockfish/stockfish-windows-x86-64-avx2.exe")
stockfish_good.set_depth(10)#How deep the AI looks
stockfish_good.set_skill_level(10)#Highest rank stockfish
import chess.pgn
import pandas as pd
pd.options.display.max_columns=999
import datetime
import tqdm
import zipfile

In [60]:
games=pd.read_csv("../../testData/loaded_games.csv")
games

,headers,moves
0,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","['e2e4', 'e7e6', 'd2d4', 'd7d5', 'b1c3', 'f8b4..."
1,"{'_tag_roster': {'Event': 'Rated Rapid game', ...","['e2e4', 'c7c5', 'g1f3', 'd7d6', 'b1c3', 'g8f6..."
2,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","['e2e4', 'g7g6', 'd2d4', 'f8g7', 'b1c3', 'c7c6..."
3,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","['d2d4', 'd7d5', 'c2c4', 'c7c6', 'b1c3', 'g8f6..."
4,{'_tag_roster': {'Event': 'Rated Blitz tournam...,"['d2d4', 'f7f5', 'g2g3', 'g8f6', 'f1g2', 'g7g6..."
...,...,...
1995,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","['e2e4', 'd7d6', 'd2d4', 'g8f6', 'b1c3', 'g7g6..."
1996,{'_tag_roster': {'Event': 'Rated Blitz tournam...,"['e2e4', 'e7e6', 'b2b3', 'd7d5', 'c1b2', 'd5e4..."
1997,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","['e2e4', 'c7c5', 'g1f3', 'd7d6', 'd2d4', 'c5d4..."
1998,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","['e2e4', 'e7e5', 'g1f3', 'b8c6', 'f1b5', 'g8f6..."


In [61]:
#Convert moves from string into list of move strings 

#Try not to run eval unless you really need to and are sure the text comes from a trusted source
games['moves']=games['moves'].apply(lambda x: eval(x))
games['headers']=games['headers'].apply(lambda x: eval(x))#to dict
games['lichess_id']=games['headers'].apply(lambda x: x.get("_others",{}).get("LichessURL","").split("/")[-1])
games['ECO']=games['headers'].apply(lambda x: x.get("_others",{}).get("ECO"))
games['white_elo']=games['headers'].apply(lambda x: x.get("_others",{}).get("WhiteElo","").split("-")[0]).astype(int)
games['black_elo']=games['headers'].apply(lambda x: x.get("_others",{}).get("BlackElo","").split("-")[0]).astype(int)
games['opening_name']=games['headers'].apply(lambda x: x.get("_others",{}).get("Opening",""))

games['white_score']=games['headers'].apply(lambda x: x.get("_tag_roster",{}).get("Result","").split("-")[0].replace("1/2",".5")).astype(float)
games

,headers,moves,lichess_id,ECO,white_elo,black_elo,opening_name,white_score
0,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[e2e4, e7e6, d2d4, d7d5, b1c3, f8b4, e4e5, g8e...",kuUOsOML,C18,2413,2254,"French Defense: Winawer Variation, Poisoned Pa...",0.5
1,"{'_tag_roster': {'Event': 'Rated Rapid game', ...","[e2e4, c7c5, g1f3, d7d6, b1c3, g8f6, g2g3, g7g...",YcFqJqyM,B50,2323,2415,Sicilian Defense,1.0
2,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[e2e4, g7g6, d2d4, f8g7, b1c3, c7c6, c1e3, d7d...",AY9kSWFt,B06,2497,2426,Modern Defense: Standard Line,0.5
3,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[d2d4, d7d5, c2c4, c7c6, b1c3, g8f6, g1f3, e7e...",ZZiTCh8x,D52,2387,2416,Queen's Gambit Declined,0.0
4,{'_tag_roster': {'Event': 'Rated Blitz tournam...,"[d2d4, f7f5, g2g3, g8f6, f1g2, g7g6, g1f3, f8g...",DHvjCHM2,A81,2288,2469,Dutch Defense: Semi-Leningrad Variation,1.0
...,...,...,...,...,...,...,...,...
1995,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[e2e4, d7d6, d2d4, g8f6, b1c3, g7g6, g1f3, f8g...",qmP74cCM,B08,2412,2424,Pirc Defense: Classical Variation,0.0
1996,{'_tag_roster': {'Event': 'Rated Blitz tournam...,"[e2e4, e7e6, b2b3, d7d5, c1b2, d5e4, b1c3, f7f...",tDWQPO0F,C00,2429,2251,"French Defense: Horwitz Attack, Papa-Ticulat G...",1.0
1997,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[e2e4, c7c5, g1f3, d7d6, d2d4, c5d4, d1d4, b8c...",wbiRGx4h,B53,2401,2464,Sicilian Defense: Chekhover Variation,1.0
1998,"{'_tag_roster': {'Event': 'Rated Blitz game', ...","[e2e4, e7e5, g1f3, b8c6, f1b5, g8f6, e1g1, f6e...",KR6JypMY,C67,2383,2462,"Ruy Lopez: Berlin Defense, l'Hermet Variation",0.0


# Running Stored Game


In [65]:
#Standard point values for different pieces. Lowercase is white piece and uppercase is black
piece_scores={
    "p": 1,
    "P": -1,
    "r": 5,
    "R": -5,
    "n": 3,
    "N": -3,
    "b": 3,
    "B": -3,
    "q": 9,
    "Q": -9
}

In [73]:
game=games.iloc[0]

All we have to do is iterate through moves. For [chessnit.com](http://www.chessnit.com) I was looking at specific positions so I simply chose a random number to evaluate in each game.

In [79]:
import random

#Used by chessnit.com
def build_stored_game_analysis(game, move_number, prev_evaluation):
    row={}
    row['move_number']=move_number
    row['taken']=[]
    board=chess.Board()
    for san in game['moves'][:move_number]:
        parsed_san=board.parse_san(san)
        taken=board.piece_at(parsed_san.to_square)
        if taken:
            row['taken'].append(taken.__str__())
        move=board.push_san(san)
    row['invalid']=bool(board.promoted) or bool(board.outcome())
    stockfish_good.set_fen_position(board.fen())
    evaluation=stockfish_good.get_evaluation()
    row['evaluation']=evaluation['value']
    row['evaluation_change']=evaluation['value']-prev_evaluation
    
    
    row['taken_score']=sum([piece_scores.get(p) for p in row['taken']])*100
    row['fen']=board.fen()
    row['url']=game['headers'].get("_others",{}).get("LichessURL","")+f"#{move_number}"
    try:
        row['last_move']=san
    except:
        print(game)
        row['invalid']=True
    return row, evaluation['value']
row, x=build_stored_game_analysis(game, 4, 0)
row
x

30

In [84]:

import pandas as pd
import tqdm

def analyze_games(games, n, max_move_number):
    all_game_analysis = []
    
    # Iterate over the first n games in the DataFrame
    prev_evaluation  = 0
    for i in tqdm.tqdm(range(min(n, len(games)))):
        game = games.iloc[i]
        game_analysis = []
        
        # Analyze each game up to the specified move number
        for move_number in range(1, max_move_number + 1):
            analysis_result, current_evaluation = build_stored_game_analysis(game, move_number,prev_evaluation)
            game_analysis.append(analysis_result)
            prev_evaluation = current_evaluation
        # Store the analysis results for each game
        all_game_analysis.append(pd.DataFrame(game_analysis).set_index("move_number"))
    
    return all_game_analysis


In [ ]:

# Define the number of games and the maximum move number you want to analyze
n_games = 2  # For example, analyze the first 10 games
max_move_number = 15

# Assuming 'games' is your DataFrame containing the games
games_analysis = analyze_games(games, n_games, max_move_number)

# Now, games_analysis is a list of DataFrames, each containing the analysis of one game


100%|██████████| 2/2 [00:05<00:00,  2.64s/it]


In [98]:
games_analysis

[                 taken  invalid  evaluation  evaluation_change  taken_score  \
 move_number                                                                   
 1                   []    False          26                 26            0   
 2                   []    False          39                 13            0   
 3                   []    False          26                -13            0   
 4                   []    False          30                  4            0   
 5                   []    False          35                  5            0   
 6                   []    False          58                 23            0   
 7                   []    False          70                 12            0   
 8                   []    False          75                  5            0   
 9                   []    False          40                -35            0   
 10                 [N]    False          44                  4         -300   
 11              [N, b]    False        

In [97]:
games_analysis[0]['evaluation_change']

move_number
1     26
2     13
3    -13
4      4
5      5
6     23
7     12
8      5
9    -35
10     4
11    40
12     4
13   -15
14    19
15   -39
Name: evaluation_change, dtype: int64